pip3 install scipy==1.13.1 && \
pip3 install -U scikit-learn && \
pip3 install scanpy==1.10.3 && \
pip3 install anndata==0.10.5.post1 && \
pip3 install einops==0.8.0 && \
pip3 install local-attention==1.9.14 && \
pip3 install gdown && \
pip3 install memory-profiler && \
pip3 installmega.py


In [20]:
%load_ext autoreload
%autoreload 2

import os
import gc
import argparse
import json
import random
import math
from functools import reduce												 
import numpy as np 
import pandas as pd
from collections import Counter
from pathlib import Path

import scipy    
import sklearn 
import scanpy as sc
import anndata as ad

import utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dir = Path('performer_pytorch')
data_dir.mkdir(exist_ok=True)  

from performer_pytorch.performer_pytorch import PerformerLM

/usr/local/lib/python3.10/site-packages/local_attention/rotary.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.10/site-packages/local_attention/rotary.py:55: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


In [3]:
filename = 'panglao_human.h5ad'
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)  
path_data = data_dir / filename

if False: 
    !gdown https://drive.google.com/uc?id=15biFWDLipYpM9iTGgaPmBhlgjkUu08bz -O {path_data} 

if False: 
    from mega import Mega
    mega = Mega()
    m = mega.login()
    file = m.download_url('https://mega.nz/file/thZVlZrC#g7WYLCIXyBSdabju3jpfLUanx_MTaB4rmhV3h1RnFcU', 
                          dest_filename=path_data)

In [4]:
path_gen2vec = data_dir / 'gene2vec_16906.npy'

if False: 
    !gdown https://drive.google.com/uc?id=15aCnM4kyTBwe6S39RsqJ8JJnE9Xj-gyi -O {path_gen2vec}

In [5]:
data = sc.read_h5ad(path_data)

print(type(data))
print(data.shape)

# Each row corresponds to a cell's gene expression data, and each column corresponds to a specific gene.
pd.DataFrame(data[:10, :17].X.toarray(), columns=data[:10, :17].var_names, index=data[:10, :17].obs_names)         

<class 'anndata._core.anndata.AnnData'>
(1357593, 16906)


/usr/local/lib/python3.10/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,A1BG,A1CF,A2ML1,A2M,A4GALT,A4GNT,AAAS,AACS,AADACL2,AADAC,AADAT,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2
09cbFwH3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.840998,0.00000,0.000000,0.0,0.000000
0AdkBIuA,0.0,0.0,0.0,0.000000,0.0,0.0,0.028411,0.028411,0.0,0.0,0.0,0.0,1.626798,0.00000,0.000000,0.0,0.000000
0frx9XJU,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.049059,0.0,0.000000
0NaEBg2H,0.0,0.0,0.0,0.453423,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.037561,0.00000,0.000000,0.0,3.388124
0NQlCyEz,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.750278,0.00000,0.016584,0.0,0.000000
0WaPBWQw,0.0,0.0,0.0,0.000000,0.0,0.0,2.452006,2.610123,0.0,0.0,0.0,0.0,0.012644,0.00000,1.919015,0.0,0.805893
12cCwzii,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.166541,0.00000,0.000000,0.0,0.000000
1iEKkCnT,0.0,0.0,0.0,0.515515,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,3.563018,0.00000,0.000000,0.0,0.000000
1lIceRcL,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.39369,0.000000,0.0,2.837606
1MhXbC7E,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.961442,0.00000,0.000000,0.0,0.000000


In [8]:
print(data.shape)
len(set(data.obs_names)), len(set(data.var_names))

(1357593, 16906)


(812188, 16906)

In [9]:
data.obs

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,Batch
09cbFwH3,2371,2371,689381.0,43521.0,6.313055,SRA275902
0AdkBIuA,2568,2568,601183.0,41842.0,6.959944,SRA275902
0frx9XJU,1570,1570,341170.0,33386.0,9.785737,SRA275902
0NaEBg2H,3141,3141,438603.0,48809.0,11.128287,SRA275902
0NQlCyEz,3289,3289,1020715.0,36329.0,3.559172,SRA275902
...,...,...,...,...,...,...
TTTTGTAATATA,794,794,1433.0,96.0,6.699232,SRA878024_SRS4660848
TTTTTCGCGCGG,609,609,988.0,71.0,7.186235,SRA878024_SRS4660848
TTTTTGGAGCCC,983,983,1930.0,102.0,5.284974,SRA878024_SRS4660848
TTTTTTAGCCCC,323,323,426.0,15.0,3.521127,SRA878024_SRS4660848


# Sort torch.distributed, CUDA


- pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --upgrade --force-reinstall
- pip3 install fsspec==2023.10.0


In [6]:
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from sklearn.model_selection import train_test_split

In [7]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA available:", torch.version.cuda) 
print(torch.distributed.is_available())

CUDA available: True
CUDA available: 11.8
True


In [15]:
import os

rank = 0
world_size = 1
is_master = rank == 0
local_rank = 0        # as running on a single GPU 

os.environ['RANK'] = str(rank)
os.environ['WORLD_SIZE'] = str(world_size) 
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

In [9]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

dist.init_process_group(backend='gloo', init_method='env://')
device = torch.device("cuda:0")
torch.cuda.set_device(device)

- Padding tokens are used to make all sequences in a batch the same length, ensuring consistency for model training. Ignored during the learning process.

In [10]:
SEED = 2021
EPOCHS = 3
BATCH_SIZE = 3
GRADIENT_ACCUMULATION = 60
LEARNING_RATE = 1e-4
SEQ_LEN = 16906 + 1
VALIDATE_EVERY = 1
CLASS = 5 + 2
MASK_PROB = 0.15
REPLACE_PROB = 0.9
RANDOM_TOKEN_PROB = 0.0
MASK_TOKEN_ID = CLASS - 1  # AK: Problem or not?  
PAD_TOKEN_ID  = CLASS - 1
MASK_IGNORE_TOKEN_IDS = [0]
POS_EMBED_USING = True

model_name = 'panglao_pretrain'
ckpt_dir = './ckpts/'

In [11]:
class SCDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        rand_start = random.randint(0, self.data.shape[0]-1)
        full_seq = self.data[rand_start].toarray()[0]        # Flattening from 2D to 1D. Takes one whole row. 
        full_seq[full_seq > (CLASS - 2)] = CLASS - 2         # Clip all values in full_seq to be at most CLASS - 2. Normalization. 
        full_seq = torch.from_numpy(full_seq).long()
        full_seq = torch.cat((full_seq, torch.tensor([0]))).to(device)
        return full_seq

    def __len__(self):
        return self.data.shape[0]

data_train, data_val = train_test_split(data.X, test_size=0.05,random_state=SEED)

train_dataset = SCDataset(data_train)
val_dataset = SCDataset(data_val)

In [12]:
train_dataset.__len__(), val_dataset.__len__()

(1289713, 67880)

In [13]:
train_sampler = DistributedSampler(train_dataset)
val_sampler   = utils.SequentialDistributedSampler(val_dataset, batch_size=BATCH_SIZE, world_size=world_size)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, sampler=val_sampler)

In [16]:
model = PerformerLM(num_tokens = CLASS, dim = 200, depth = 6, max_seq_len = SEQ_LEN,heads = 10,local_attn_heads = 0, 
                    g2v_position_emb = POS_EMBED_USING)
model.to(device)
model = DDP(model, device_ids=[local_rank], output_device=local_rank) # DistributedDataParallel

In [17]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

# learning rate scheduler
scheduler = utils.CosineAnnealingWarmupRestarts(optimizer, first_cycle_steps=15, cycle_mult=2, max_lr=LEARNING_RATE,
                                                min_lr=1e-6, warmup_steps=5,gamma=0.9)

loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_ID, reduction='mean').to(local_rank)
softmax = nn.Softmax(dim=-1)

In [18]:
# get the random prob matrix and True means smaller than prob threshold
def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob

# get the mask matrix which cannot be masked
def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask

def get_mask_subset_with_prob(mask, prob):

    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)          # num of mask of a single sequence in average
    num_tokens = mask.sum(dim=-1, keepdim=True)     # num of pure tokens of each sequence except special tokens
    mask_excess = torch.cat((torch.zeros(0), torch.arange(mask.size(-1)).repeat(mask.size(0)))).reshape(mask.size(0),mask.size(-1)).to(device)
    mask_excess = (mask_excess >= (num_tokens * prob).ceil())        # only 15% of pure tokens can be masked
    mask_excess = mask_excess[:, :max_masked]       # get difference between 15% of pure tokens and 15% of all tokens
    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)     # rand (0-1) as prob, special token use -1e9
    _, sampled_indices = rand.topk(max_masked, dim=-1)      # get index of topk prob to mask
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)        # delete difference of mask not pure
    new_mask = torch.zeros((batch, seq_len + 1), device=device)     # get (batch, seq_len) shape zero matrix
    new_mask.scatter_(-1, sampled_indices, 1)       # set masks in zero matrix as 1
    
    return new_mask[:, 1:].bool()       # the final mask, True is mask

def data_mask(data, mask_prob = MASK_PROB, replace_prob = REPLACE_PROB, num_tokens = None, random_token_prob = RANDOM_TOKEN_PROB,
              mask_token_id = MASK_TOKEN_ID, pad_token_id = PAD_TOKEN_ID, mask_ignore_token_ids = MASK_IGNORE_TOKEN_IDS):
    
    mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])
    # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
    # also do not include these special tokens in the tokens chosen at random
    no_mask = mask_with_tokens(data, mask_ignore_token_ids)    # ignore_token as True, will not be masked later
    mask = get_mask_subset_with_prob(~no_mask, mask_prob)      # get the True/False mask matrix
    # get mask indices
    ## mask_indices = torch.nonzero(mask, as_tuple=True)   # get the index of mask(nonzero value of mask matrix)
    # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
    masked_input = data.clone().detach()
    # if random token probability > 0 for mlm
    if random_token_prob > 0:
        assert num_tokens is not None, 'num_tokens keyword must be supplied when instantiating MLM if using random token replacement'
        random_token_prob = prob_mask_like(data, random_token_prob)       # get the mask matrix of random token replace
        random_tokens = torch.randint(0, num_tokens, data.shape, device=data.device)     # generate random token matrix with the same shape as input
        random_no_mask = mask_with_tokens(random_tokens, mask_ignore_token_ids)        # not masked matrix for the random token matrix
        random_token_prob &= ~random_no_mask        # get the pure mask matrix of random token replace
        random_indices = torch.nonzero(random_token_prob, as_tuple=True)        # index of random token replace
        masked_input[random_indices] = random_tokens[random_indices]        # replace some tokens by random token
    # [mask] input
    replace_prob = prob_mask_like(data, replace_prob)     # get the mask matrix of token being masked
    masked_input = masked_input.masked_fill(mask * replace_prob, mask_token_id)        # get the data has been masked by mask_token
    # mask out any tokens to padding tokens that were not originally going to be masked
    labels = data.masked_fill(~mask, pad_token_id)        # the label of masked tokens
    return masked_input, labels

In [ ]:
dist.barrier()
for i in range(1, EPOCHS+1):
    train_loader.sampler.set_epoch(i)
    model.train()
    dist.barrier()
    running_loss = 0.0
    cum_acc = 0.0
    for index, data in enumerate(train_loader):
        index += 1
        data = data.to(device)
        
        data, labels = data_mask(data)
        
        # accumulates gradients without synchronization, saving resources
        if index % GRADIENT_ACCUMULATION != 0: 
            with model.no_sync():
                logits = model(data)
                loss = loss_fn(logits.transpose(1, 2), labels) / GRADIENT_ACCUMULATION
                loss.backward()
        
        # synchronizes gradients, applies updates, and resets for the next accumulation phase
        if index % GRADIENT_ACCUMULATION == 0: 
            logits = model(data)
            loss = loss_fn(logits.transpose(1, 2), labels) / GRADIENT_ACCUMULATION
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), int(1e2)) # clip grads to prevent exploding gradients
            optimizer.step()                                             # updates the model parameters based on the accumulated gradients
            optimizer.zero_grad()                                        # resets the gradients to zero, preparing for the next accumulation
        running_loss += loss.item()
        final = softmax(logits)[..., 1:-1]
        final = final.argmax(dim=-1) + 1
        pred_num = (labels != PAD_TOKEN_ID).sum(dim=-1)
        correct_num = ((labels != PAD_TOKEN_ID) * (final == labels)).sum(dim=-1)
        cum_acc += torch.true_divide(correct_num, pred_num).mean().item()
    
    epoch_loss = running_loss / index
    epoch_acc = 100 * cum_acc / index
    epoch_loss = utils.get_reduced(epoch_loss, local_rank, 0, world_size)
    epoch_acc  = utils.get_reduced(epoch_acc, local_rank, 0, world_size)
    if is_master:
        print(f'    ==  Epoch: {i} | Training Loss: {epoch_loss:.6f} | Accuracy: {epoch_acc:6.4f}%  ==')
    dist.barrier()
    scheduler.step()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        dist.barrier()
        running_loss = 0.0
        running_error = 0.0
        predictions = []
        truths = []
        with torch.no_grad():
            for index, data in enumerate(val_loader):
                index += 1
                data = data.to(device)
                
                data, labels = data_mask(data)

                logits = model(data)
                loss = loss_fn(logits.transpose(1, 2), labels)
                running_loss += loss.item()
                softmax = nn.Softmax(dim=-1)
                final = softmax(logits)[..., 1:-1]
                final = final.argmax(dim=-1) + 1
                predictions.append(final)
                truths.append(labels)
            del data, labels, logits, final
            # gather
            predictions = utils.distributed_concat(torch.cat(predictions, dim=0), len(val_sampler.dataset), world_size)
            truths = utils.distributed_concat(torch.cat(truths, dim=0), len(val_sampler.dataset), world_size)
            correct_num = ((truths != PAD_TOKEN_ID) * (predictions == truths)).sum(dim=-1)[0].item()
            val_num = (truths != PAD_TOKEN_ID).sum(dim=-1)[0].item()
            val_loss = running_loss / index
            val_loss = utils.get_reduced(val_loss, local_rank, 0, world_size)
        if is_master:
            val_acc = 100 * correct_num / val_num
            print(f'    ==  Epoch: {i} | Validation Loss: {val_loss:.6f} | Accuracy: {val_acc:6.4f}%  ==')
    del predictions, truths

    if is_master:
        utils.save_ckpt(i, model, optimizer, scheduler, epoch_loss, model_name, ckpt_dir)